In [1]:
### Import libraries

import pandas as pd
import numpy as np
import pickle as pkl # Library for saving location data

#import Google maps API through geopy. 
from geopy.geocoders import GoogleV3
api_key = "your_api_key_here" # Placeholder for Google Maps API key
geolocator = GoogleV3(api_key = api_key) 

#import Google maps for distance calculations
import googlemaps
gmaps = googlemaps.Client(key=api_key) 

#Import country converter (if countries are in ISO-2 format)
import country_converter as coco
cc = coco.CountryConverter()

# Import pickle dictionary, if it exists, otherwise create a blank dictionary. 
try:
    searchterms_dict = pkl.load(open("searchterms.pkl","rb"))
except:
    print("No pickle dictionary found, creating blank")
    searchterms_dict = dict()
    
# Initialise API use counter
api_counter = 0

# Import your data

In [6]:
### Import file and do some basic data cleaning

#Load csv file from current working directory. For Excel files, use pd.read_excel
Mydata = pd.read_csv('Location data.csv', index_col = 'ID').dropna(how = "all") #Import file and remove any completely empty rows

#Remove any white spaces in column titles
Mydata.columns = [column_name.strip() for column_name in Mydata.columns] 
#Convert column titles to title case and replace spaces with underscores
Mydata.columns = Mydata.columns.str.title().str.replace(" ","_")

#Import country columns as category (this may improve performance in large datasets) and remove white spaces
Mydata["Origin_Country"] = Mydata["Origin_Country"].astype("category").str.strip()
o_country = Mydata.groupby("Origin_Country") 
Mydata["Destination_Country"] = Mydata["Destination_Country"].astype("category").str.strip()
d_country = Mydata.groupby("Destination_Country") 

#Import postal code columns as string and remove white spaces
#It's good to specify postal codes as the string data type. 
#Postal codes can sometimes be interpreted as int or float by Python, which can lead to problems, e.g. with removal of leading zeros (e.g. postal code 00100 becomes 100)
Mydata["Origin_Postal"] = Mydata["Origin_Postal"].astype("str").str.strip().str.upper()
Mydata["Destination_Postal"] = Mydata["Destination_Postal"].astype("str").str.strip().str.upper()

# Convert city columns to title case and remove white spaces: 
Mydata["Origin_City"] = Mydata["Origin_City"].str.title().str.strip()
Mydata["Destination_City"] = Mydata["Destination_City"].str.title().str.strip()

# Optional: remove unreadable characters from city columns
Mydata['Destination_City'] = Mydata['Destination_City'].str.replace(' ','_').str.replace(r'\W+','', regex=True).str.replace('_',' ')
Mydata['Origin_City'] = Mydata['Origin_City'].str.replace(' ','_').str.replace(r'\W+','', regex=True).str.replace('_',' ')


#view the dataframe
Mydata 

,Origin_Country,Origin_City,Origin_Postal,Destination_Country,Destination_City,Destination_Postal
ID,,,,,,
1,IT,Roma,00100,ES,Barcelona,08001
2,FR,Paris,75005,ES,Zaragoza,50001
3,FR,Brest,29200,ES,Barcelona,08001
4,NL,Rotterdam,3045,AT,Innsbruck,6020
5,NL,Rotterdam,3045,DE,Düsseldorf,40210
6,NL,Rotterdam,3045,BE,Gent,9040
7,FI,Helsinki,00100,FI,Jyväskylä,40100
8,IT,Roma,00100,ES,Zaragoza,50001
9,IT,Roma,00100,DE,Düsseldorf,40210


# Find coordinates of origins and destinations

## Destination coordinates

In [4]:
### Create dataframe consisting of unique destinations. 

Destinations = Mydata[['Destination_Country', 'Destination_Postal', 'Destination_City']].drop_duplicates()
print('Rows in dataset: ', len(Mydata))
print('Unique destinations:', len(Destinations))

Rows in dataset:  17
Unique destinations: 10


In [7]:
### Find Destination coordinates 

for i in Destinations.index:    
    postal = Destinations.loc[i,'Destination_Postal']
    city = Destinations.loc[i,'Destination_City']
    country = Destinations.loc[i,'Destination_Country']
# Convert ISO-2 country codes to full country names 
    try: 
        country = cc.convert(country, to = 'name_short') 
    except:
        country = Destinations.loc[i,'Destination_Country'] 

    
### Determine the search term (essentially, what you would type into Google Maps to search for an address)
    SearchTerm = f'{city} {postal} {country}'

# Check if searchterm is in dictionary. If not, use API and add to dictionary.
    if SearchTerm in searchterms_dict:
        location = searchterms_dict[SearchTerm]
    else:
        location = geolocator.geocode(SearchTerm, timeout = 100)
        api_counter += 1
        searchterms_dict[SearchTerm] = location
###If the search term doesn't yield a valid search result, try searching for city & country
    if not location: 
        print(f"First attempt unsuccessful: {SearchTerm}")
        SearchTerm = f'{city} {country}'
        #Check if searchterm is in dictionary.
        if SearchTerm in searchterms_dict:
            location = searchterms_dict[SearchTerm]
        else:
            location = geolocator.geocode(SearchTerm, timeout = 100)
            api_counter += 1
            # Add to dictionary
            searchterms_dict[SearchTerm] = location
        if not location: #If that didn't work, continue to next iteration
            print(f"Second attempt unsuccessful: {SearchTerm}")
            continue
            
### Get coordinates: 
    try: 
        Lat = location.latitude
    except:
        print(f'No coordinates: {SearchTerm}')
        continue
    Lon = location.longitude    
        
### Add the coordinates to the dataframe: 
    Destinations.loc[i, 'Destination_Lat'] = Lat
    Destinations.loc[i, 'Destination_Lon'] = Lon
    
        
### Optional: Get individual address components from the search result (country, city, postal code)
    address_components = location.raw['address_components']
#Country ISO-2 code
    country_short = [addr['short_name'] for addr in address_components if 'country' in addr['types']]
    if not country_short: #If no country is found, return NaN and skip to next iteration
        country_short = np.nan
        print("No country found: ", SearchTerm)
        continue
#Country name
    country_long = [addr['long_name'] for addr in address_components if 'country' in addr['types']]
    if not country_long: 
        country_long = np.nan
#City name
    gen_city = [addr['long_name'] for addr in address_components if 'locality' in addr['types']]
    if not gen_city: #If no city is found, return NaN
        gen_city = np.nan
#Postal code
    gen_postal = [addr['short_name'] for addr in address_components if 'postal_code' in addr['types']]
    if not gen_postal: 
        gen_postal = np.nan
#Full address
    try:     
        Address = location.address
    except: 
        Address = np.nan
        
###Optional columns to add to the dataframe
    Destinations.loc[i, 'Destination_SearchTerm'] = SearchTerm
    Destinations.loc[i, 'Gen_Destination_Address'] = Address
    Destinations.loc[i, 'Gen_Destination_ISO'] = country_short
    Destinations.loc[i, 'Gen_Destination_Country'] = country_long
    Destinations.loc[i, 'Gen_Destination_City'] = gen_city
    Destinations.loc[i, 'Gen_Destination_Postal'] = gen_postal   

print(f'Api counter use this time: {api_counter}')

Api counter use this time: 0


In [8]:
Destinations

,Destination_Country,Destination_Postal,Destination_City,Destination_Lat,Destination_Lon,Destination_SearchTerm,Gen_Destination_Address,Gen_Destination_ISO,Gen_Destination_Country,Gen_Destination_City,Gen_Destination_Postal
ID,,,,,,,,,,,
1,ES,08001,Barcelona,41.379586,2.168357,Barcelona 08001 Spain,"08001 Barcelona, Spain",ES,Spain,Barcelona,08001
2,ES,50001,Zaragoza,41.648968,-0.878239,Zaragoza 50001 Spain,"50001 Zaragoza, Spain",ES,Spain,Zaragoza,50001
4,AT,6020,Innsbruck,47.269258,11.404079,Innsbruck 6020 Austria,"6020 Innsbruck, Austria",AT,Austria,Innsbruck,6020
5,DE,40210,Düsseldorf,51.222529,6.787720,Düsseldorf 40210 Germany,"40210 Düsseldorf, Germany",DE,Germany,Düsseldorf,40210
6,BE,9040,Gent,51.064336,3.756635,Gent 9040 Belgium,"9040 Ghent, Belgium",BE,Belgium,Ghent,9040
7,FI,40100,Jyväskylä,62.239852,25.756001,Jyväskylä 40100 Finland,"40100 Jyväskylä sub-region, Finland",FI,Finland,Jyväskylä,40100
11,CZ,118 00,Praha,50.088054,14.402798,Praha 118 00 Czech Republic,"118 00 Prague-Prague 1, Czechia",CZ,Czechia,NaN,118 00
14,GB,BA1 0FA,Bath,51.378858,-2.355562,Bath BA1 0FA United Kingdom,"Bath BA1 0FA, UK",GB,United Kingdom,NaN,BA1 0FA
15,GB,CB1 0BD,Cambridge,52.209345,0.148231,Cambridge CB1 0BD United Kingdom,"Cambridge CB1 0BD, UK",GB,United Kingdom,NaN,CB1 0BD


In [8]:
### Optional: Export your destination coordinates to Excel (for csv, use .to_csv())
Destinations.to_excel('Destinations.xlsx', index = False)
### Save updated searchterms_dict
pkl.dump(searchterms_dict, open("searchterms.pkl", "wb"))

## Origin coordinates

In [9]:
#Create dataframe consisting of unique origins. 
Origins = Mydata[['Origin_Country', 'Origin_Postal', 'Origin_City']].drop_duplicates()
print('Rows in dataset: ', len(Mydata))
print('Unique origins:', len(Origins))

Rows in dataset:  17
Unique origins: 8


In [10]:
### Find Origin coordinates with duplicates removed
### This is basically the same script as used for destinations

## Optional, reset api_counter: 
api_counter = 0

for i in Origins.index:
    postal = Origins.loc[i,'Origin_Postal']
    city = Origins.loc[i,'Origin_City']
    country = Origins.loc[i,'Origin_Country']
    try: #convert ISO name to full country name with country converter
        country = cc.convert(country, to = 'name_short') 
    except: 
         country = Origins.loc[i,'Origin_Country']

### Determine the search term (essentially, what you would type into Google Maps to search for an address)
    SearchTerm = f'{city} {postal} {country}'
    
# Check if searchterm is in dictionary. If not, use API and add to dictionary.
    if SearchTerm in searchterms_dict:
        location = searchterms_dict[SearchTerm]
### Search for the location using the previously defined search term    
    else:
        location = geolocator.geocode(SearchTerm, timeout = 100)
        api_counter += 1
        searchterms_dict[SearchTerm] = location
###If the search term doesn't yield a valid search result, try searching for city & country
    if not location: 
        print(f"First attempt unsuccessful: {SearchTerm}")
        SearchTerm = f'{city} {country}'
        #Check if searchterm is in dictionary.
        if SearchTerm in searchterms_dict:
            location = searchterms_dict[SearchTerm]
        else:
            location = geolocator.geocode(SearchTerm, timeout = 100)
            api_counter += 1
            # Add to dictionary
            searchterms_dict[SearchTerm] = location
        if not location: #If that didn't work, continue to next iteration
            print(f"Second attempt unsuccessful: {SearchTerm}")
            continue

            
### Get coordinates: 
    try: 
        Lat = location.latitude
    except:
        print(f'No coordinates: {SearchTerm}')        
        continue
    Lon = location.longitude    
    
###Add the coordinates to the dataframe: 
    Destinations.loc[i, 'Origin_Lat'] = Lat
    Destinations.loc[i, 'Origin_Lon'] = Lon
    
    
### Optional: Get individual address components from the search result (country, city, postal code)
    address_components = location.raw['address_components']
#Country ISO-2 code
    country_short = [addr['short_name'] for addr in address_components if 'country' in addr['types']]
    if not country_short: #If no country is found, return NaN and skip to next iteration
        country_short = np.nan
        print("No country found: ", SearchTerm)
        continue
#Country name
    country_long = [addr['long_name'] for addr in address_components if 'country' in addr['types']]
    if not country_long: 
        country_long = np.nan
#City name
    gen_city = [addr['long_name'] for addr in address_components if 'locality' in addr['types']]
    if not gen_city: #If no city is found, return NaN
        gen_city = np.nan
#Postal code
    gen_postal = [addr['short_name'] for addr in address_components if 'postal_code' in addr['types']]
    if not gen_postal: 
        gen_postal = np.nan
#Full address
    try:     
        Address = location.address
    except: 
        Address = np.nan
        
###Optional columns to add to the dataframe 
    Origins.loc[i, 'Origin_SearchTerm'] = SearchTerm
    Origins.loc[i, 'Origin_Lat'] = Lat
    Origins.loc[i, 'Origin_Lon'] = Lon
    Origins.loc[i, 'Gen_Origin_Address'] = Address
    Origins.loc[i, 'Gen_Origin_ISO'] = country_short
    Origins.loc[i, 'Gen_Origin_Country'] = country_long
    Origins.loc[i, 'Gen_Origin_City'] = gen_city
    Origins.loc[i, 'Gen_Origin_Postal'] = gen_postal   
    
print(f'Api counter use this time: {api_counter}')

Api counter use this time: 7


In [11]:
Origins

,Origin_Country,Origin_Postal,Origin_City,Origin_SearchTerm,Origin_Lat,Origin_Lon,Gen_Origin_Address,Gen_Origin_ISO,Gen_Origin_Country,Gen_Origin_City,Gen_Origin_Postal
ID,,,,,,,,,,,
1,IT,00100,Roma,Roma 00100 Italy,41.922837,12.501430,"00100 Rome, Metropolitan City of Rome, Italy",IT,Italy,Rome,00100
2,FR,75005,Paris,Paris 75005 France,48.840853,2.351848,"5th arrondissement of Paris, 75005 Paris, France",FR,France,Paris,75005
3,FR,29200,Brest,Brest 29200 France,48.401220,-4.492127,"29200 Brest, France",FR,France,Brest,29200
4,NL,3045,Rotterdam,Rotterdam 3045 Netherlands,51.953491,4.450397,"3045 Rotterdam, Netherlands",NL,Netherlands,Rotterdam,3045
7,FI,00100,Helsinki,Helsinki 00100 Finland,60.172667,24.932009,"00100 Helsinki, Finland",FI,Finland,Helsinki,00100
12,AT,6020,Innsbruck,Innsbruck 6020 Austria,47.269258,11.404079,"6020 Innsbruck, Austria",AT,Austria,Innsbruck,6020
13,PL,60-119,Poznań,Poznań 60-119 Poland,52.381617,16.885076,"60-119 Poznań, Poland",PL,Poland,Poznań,60-119
14,GB,BS1 1EE,Bristol,Bristol BS1 1EE United Kingdom,51.454773,-2.595731,"St Stephen's St, Bristol BS1 1EE, UK",GB,United Kingdom,NaN,BS1 1EE


In [12]:
### Optional: Export your origin coordinates to Excel (for csv, use .to_csv())
Origins.to_excel('Origins.xlsx', index = False)
# Save updated searchterms_dict to file
pkl.dump(searchterms_dict, open("searchterms.pkl", "wb"))
print(f'Api counter use this time: {api_counter}')

Api counter use this time: 7


In [11]:
#View the values that have been stored in your searchterms dictionary. 
searchterms_dict#.values()

{'Barcelona 08001 Spain': Location(08001 Barcelona, Spain, (41.3795863, 2.1683571, 0.0)),
 'Zaragoza 50001 Spain': Location(50001 Zaragoza, Spain, (41.6489683, -0.8782392, 0.0)),
 'Innsbruck 6020 Austria': Location(6020 Innsbruck, Austria, (47.269258, 11.4040792, 0.0)),
 'Düsseldorf 40210 Germany': Location(40210 Düsseldorf, Germany, (51.2225286, 6.787720299999999, 0.0)),
 'Gent 9040 Belgium': Location(9040 Ghent, Belgium, (51.0643361, 3.7566349, 0.0)),
 'Jyväskylä 40100 Finland': Location(40100 Jyväskylä sub-region, Finland, (62.2398524, 25.7560014, 0.0)),
 'Praha 118 00 Czech Republic': Location(118 00 Prague-Prague 1, Czechia, (50.0880538, 14.4027976, 0.0)),
 'Bath BA1 0FA United Kingdom': Location(Bath BA1 0FA, UK, (51.3788579, -2.3555617, 0.0)),
 'Cambridge CB1 0BD United Kingdom': Location(Cambridge CB1 0BD, UK, (52.2093451, 0.1482309, 0.0)),
 'York YO1  0EB United Kingdom': Location(York YO1 0EB, UK, (53.9604297, -1.092336, 0.0))}

## Add coordinates to dataset

In [11]:
#Optional: Import Excel files with coordinates
Origins = pd.read_excel('Origins.xlsx')
Destinations = pd.read_excel('Destinations.xlsx')

In [13]:
#Add origin coordinates to the main dataset
cols = ['Origin_Country', 'Origin_Postal', 'Origin_City','Origin_Lat', 'Origin_Lon']
My_merged_data = pd.merge(Mydata, Origins[cols], how = 'left', on=['Origin_Country', 'Origin_Postal', 'Origin_City'])
My_merged_data

,Origin_Country,Origin_City,Origin_Postal,Destination_Country,Destination_City,Destination_Postal,Origin_Lat,Origin_Lon
0,IT,Roma,00100,ES,Barcelona,08001,41.922837,12.501430
1,FR,Paris,75005,ES,Zaragoza,50001,48.840853,2.351848
2,FR,Brest,29200,ES,Barcelona,08001,48.401220,-4.492127
3,NL,Rotterdam,3045,AT,Innsbruck,6020,51.953491,4.450397
4,NL,Rotterdam,3045,DE,Düsseldorf,40210,51.953491,4.450397
5,NL,Rotterdam,3045,BE,Gent,9040,51.953491,4.450397
6,FI,Helsinki,00100,FI,Jyväskylä,40100,60.172667,24.932009
7,IT,Roma,00100,ES,Zaragoza,50001,41.922837,12.501430
8,IT,Roma,00100,DE,Düsseldorf,40210,41.922837,12.501430
9,IT,Roma,00100,BE,Gent,9040,41.922837,12.501430


In [14]:
#Add destination coordinates to the dataset
cols = ['Destination_Lat', 'Destination_Lon','Destination_Country','Destination_Postal','Destination_City']
My_merged_data2 = pd.merge(My_merged_data, Destinations[cols], how = 'left', on=['Destination_Country','Destination_Postal','Destination_City'])
My_merged_data2

,Origin_Country,Origin_City,Origin_Postal,Destination_Country,Destination_City,Destination_Postal,Origin_Lat,Origin_Lon,Destination_Lat,Destination_Lon
0,IT,Roma,00100,ES,Barcelona,08001,41.922837,12.501430,41.379586,2.168357
1,FR,Paris,75005,ES,Zaragoza,50001,48.840853,2.351848,41.648968,-0.878239
2,FR,Brest,29200,ES,Barcelona,08001,48.401220,-4.492127,41.379586,2.168357
3,NL,Rotterdam,3045,AT,Innsbruck,6020,51.953491,4.450397,47.269258,11.404079
4,NL,Rotterdam,3045,DE,Düsseldorf,40210,51.953491,4.450397,51.222529,6.787720
5,NL,Rotterdam,3045,BE,Gent,9040,51.953491,4.450397,51.064336,3.756635
6,FI,Helsinki,00100,FI,Jyväskylä,40100,60.172667,24.932009,62.239852,25.756001
7,IT,Roma,00100,ES,Zaragoza,50001,41.922837,12.501430,41.648968,-0.878239
8,IT,Roma,00100,DE,Düsseldorf,40210,41.922837,12.501430,51.222529,6.787720
9,IT,Roma,00100,BE,Gent,9040,41.922837,12.501430,51.064336,3.756635


## Save dataset with coordinates as csv file

In [15]:
### Output coordinates as CSV file. 
#This allows you to check the coordinates in another programme (e.g. a visualisation tool) and manually correct any errors. 

My_merged_data2.to_csv('Data_with_coordinates.csv')

---

# Calculate distances between coordinates

In [16]:
### Read the coordinates csv file 
Mydata2 = pd.read_csv('Data_with_coordinates.csv', index_col = 0) #Load the csv file from cwd and save it as a variable. 

#Import these columns as category and remove white spaces: 
Mydata2["Origin_Country"] = Mydata2["Origin_Country"].astype("category").str.strip()
Mydata2["Destination_Country"] = Mydata2["Destination_Country"].astype("category").str.strip()

#Import postal code fields as string and remove white spaces: 
Mydata2["Origin_Postal"] = Mydata2["Origin_Postal"].astype("str").str.strip().str.upper()
Mydata2["Destination_Postal"] = Mydata2["Destination_Postal"].astype("str").str.strip().str.upper()

# Convert city columns to title case and remove white spaces: 
Mydata2["Origin_City"] = Mydata2["Origin_City"].str.title().str.strip()
Mydata2["Destination_City"] = Mydata2["Destination_City"].str.title().str.strip()

#View dataframe
Mydata2

,Origin_Country,Origin_City,Origin_Postal,Destination_Country,Destination_City,Destination_Postal,Origin_Lat,Origin_Lon,Destination_Lat,Destination_Lon
0,IT,Roma,00100,ES,Barcelona,08001,41.922837,12.501430,41.379586,2.168357
1,FR,Paris,75005,ES,Zaragoza,50001,48.840853,2.351848,41.648968,-0.878239
2,FR,Brest,29200,ES,Barcelona,08001,48.401220,-4.492127,41.379586,2.168357
3,NL,Rotterdam,3045,AT,Innsbruck,6020,51.953491,4.450397,47.269258,11.404079
4,NL,Rotterdam,3045,DE,Düsseldorf,40210,51.953491,4.450397,51.222529,6.787720
5,NL,Rotterdam,3045,BE,Gent,9040,51.953491,4.450397,51.064336,3.756635
6,FI,Helsinki,00100,FI,Jyväskylä,40100,60.172667,24.932009,62.239852,25.756001
7,IT,Roma,00100,ES,Zaragoza,50001,41.922837,12.501430,41.648968,-0.878239
8,IT,Roma,00100,DE,Düsseldorf,40210,41.922837,12.501430,51.222529,6.787720
9,IT,Roma,00100,BE,Gent,9040,41.922837,12.501430,51.064336,3.756635


In [17]:
#Remove duplicate origin-destination pairs
coor = 'Origin_City Origin_Lat Origin_Lon Destination_City Destination_Lat Destination_Lon'.split()
Coordinates = Mydata2[coor]
Coordinates = Coordinates.drop_duplicates('Origin_Lat Origin_Lon Destination_Lat Destination_Lon'.split())
print('Whole dataset: ', len(Mydata2))
print('With duplicates removed:',len(Coordinates))

Whole dataset:  17
With duplicates removed: 16


In [18]:
Coordinates

,Origin_City,Origin_Lat,Origin_Lon,Destination_City,Destination_Lat,Destination_Lon
0,Roma,41.922837,12.501430,Barcelona,41.379586,2.168357
1,Paris,48.840853,2.351848,Zaragoza,41.648968,-0.878239
2,Brest,48.401220,-4.492127,Barcelona,41.379586,2.168357
3,Rotterdam,51.953491,4.450397,Innsbruck,47.269258,11.404079
4,Rotterdam,51.953491,4.450397,Düsseldorf,51.222529,6.787720
5,Rotterdam,51.953491,4.450397,Gent,51.064336,3.756635
6,Helsinki,60.172667,24.932009,Jyväskylä,62.239852,25.756001
7,Roma,41.922837,12.501430,Zaragoza,41.648968,-0.878239
8,Roma,41.922837,12.501430,Düsseldorf,51.222529,6.787720
9,Roma,41.922837,12.501430,Gent,51.064336,3.756635


In [19]:
#Retrieve origin and destination coordinates
for i in Coordinates.index:
    ori = Coordinates.loc[i,'Origin_City']
    dest = Coordinates.loc[i,'Destination_City']
    
    olat = str(Coordinates.loc[i,'Origin_Lat'])
    olon = str(Coordinates.loc[i,'Origin_Lon'])
    dlat = str(Coordinates.loc[i,'Destination_Lat'])
    dlon = str(Coordinates.loc[i,'Destination_Lon'])
    SearchTerm1 = f'{olat} {olon}'
    SearchTerm2 = f'{dlat} {dlon}'
    
### Search for the driving distance between coordinates    
    try: 
        dist = gmaps.distance_matrix(SearchTerm1, SearchTerm2, mode="driving")
### If no distance can be calculated, continue to next iteration         
    except: 
        print(f'No distance could be calculated from {ori} to {dest}')
        continue
        
### Retrieve the driving distance in metres
    try:  
        dist_metres = dist['rows'][0]['elements'][0]['distance']['value']
    except:
        print(f'No distance could be calculated from {ori} to {dest}')
        continue
        
### Calculate distance in km
    dist_km = dist_metres / 1000
### Add distance in km to the dataframe
    Coordinates.loc[i, 'Distance'] = dist_km 

In [20]:
cols = ['Origin_Lon', 'Origin_Lat', 'Destination_Lat', 'Destination_Lon', 'Distance']
Distances = pd.merge(Mydata2, Coordinates[cols], how='left', 
                     on='Origin_Lat Origin_Lon Destination_Lat Destination_Lon'.split())

In [21]:
Distances

,Origin_Country,Origin_City,Origin_Postal,Destination_Country,Destination_City,Destination_Postal,Origin_Lat,Origin_Lon,Destination_Lat,Destination_Lon,Distance
0,IT,Roma,00100,ES,Barcelona,08001,41.922837,12.501430,41.379586,2.168357,1365.494
1,FR,Paris,75005,ES,Zaragoza,50001,48.840853,2.351848,41.648968,-0.878239,1075.871
2,FR,Brest,29200,ES,Barcelona,08001,48.401220,-4.492127,41.379586,2.168357,1278.023
3,NL,Rotterdam,3045,AT,Innsbruck,6020,51.953491,4.450397,47.269258,11.404079,944.693
4,NL,Rotterdam,3045,DE,Düsseldorf,40210,51.953491,4.450397,51.222529,6.787720,239.046
5,NL,Rotterdam,3045,BE,Gent,9040,51.953491,4.450397,51.064336,3.756635,152.926
6,FI,Helsinki,00100,FI,Jyväskylä,40100,60.172667,24.932009,62.239852,25.756001,269.042
7,IT,Roma,00100,ES,Zaragoza,50001,41.922837,12.501430,41.648968,-0.878239,1650.700
8,IT,Roma,00100,DE,Düsseldorf,40210,41.922837,12.501430,51.222529,6.787720,1434.614
9,IT,Roma,00100,BE,Gent,9040,41.922837,12.501430,51.064336,3.756635,1520.787


## Save dataset with distances as Excel file

In [23]:
# Caution: overwrites any file with the same name! 
Distances.to_excel('Data_with_distances.xlsx')